## UCI Bank Marketing 

Загрузим данные [UCI Bank Marketing Dataset](https://archive.ics.uci.edu/ml/datasets/bank+marketing). Этот датасет содержит информацию о банковском телефонном маркетинге, объектом в нем является телефонный звонок потенциальному клиенту с предложением некоторой услуги (утверждается, что это краткосрочный депозит), целевой переменной - ответ клиента (согласился ли он открыть депозит?). В качестве признакового описания используются характеристики клиента (образование, брак и т.д.), данные о звонке и различные экономические индикаторы - более подробная информация представлена в файле `bank-additional-names.txt`. Попробуем применить различные варианты работы с категориальными переменными и посмотрим какие результаты дают эти методы.

### Оглавление
* OrdinaryEncoder
* OneHotEncoder
* MeanEncoder
* SmoothMeanEncoder
* Results
* RandomForest and OneHotEncoder

In [208]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.metrics import average_precision_score, plot_precision_recall_curve, recall_score, accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, f1_score, roc_auc_score, roc_curve, plot_roc_curve
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline, Pipeline
import time

%matplotlib inline

In [209]:
df = pd.read_csv('bank-additional-full.csv',sep = ';')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [210]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

Пропущенных значений нет, почти половина признаков являются категориальными

In [211]:
df[df['y'] == 'yes']['y'].count()

4640

In [212]:
df[df['y'] == 'no']['y'].count()

36548

In [213]:
df.shape

(41188, 21)

### OrdinaryEncoder 

Попробуем закодировать категориальные признаки с помощью OrdinaryEncoder и посмотрим на метрики качества

In [214]:
y = df['y'].map({'no': 0, 'yes': 1})
X = df.drop(columns = ['y'])

In [215]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=777)

In [216]:
categorical_features = list(x_train.select_dtypes(include='object').columns)
numeric_features = list(x_train.select_dtypes(include= ['int64', 'float64']).columns)

In [217]:
models_metrics = {}

In [218]:
%%time
column_transformer = make_column_transformer((OrdinalEncoder(), categorical_features), remainder = 'passthrough')
column_transformer.fit_transform(x_train)

pipe = make_pipeline(column_transformer, LogisticRegression(max_iter = 10000))
pipe.fit(x_train, y_train)

Wall time: 5.34 s


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'month', 'day_of_week',
                                                   'poutcome'])])),
                ('logisticregression', LogisticRegression(max_iter=10000))])

In [219]:
y_pred = pipe.predict(x_test)

In [220]:
confusion = confusion_matrix(y_test, y_pred)
confusion

array([[8831,  228],
       [ 735,  503]], dtype=int64)

In [221]:
TP = confusion[1,1]
TN = confusion[0,0]
FP = confusion[0,1]
FN = confusion[1,0]

Метрики качества

In [222]:
print("recall - %.5f" %recall_score(y_test, y_pred))
print("precision - %.5f" %precision_score(y_test, y_pred))
print("f1 - %.5f" %f1_score(y_test, y_pred))
print("roc-auc score - %.5f" %roc_auc_score(y_test, y_pred))

recall - 0.40630
precision - 0.68810
f1 - 0.51092
roc-auc score - 0.69057


In [223]:
oenc = []
oenc.append(recall_score(y_test, y_pred))
oenc.append(precision_score(y_test, y_pred))
oenc.append(f1_score(y_test, y_pred))
oenc.append(roc_auc_score(y_test, y_pred))
models_metrics['OrdinaryEncoder'] = oenc

### OneHotEncoder

Закодируем признаки с помощью OneHotEncoder, посмотрим как изменится качество

In [224]:
%%time
column_transformer = make_column_transformer((OneHotEncoder(), categorical_features), remainder = 'passthrough')
column_transformer.fit_transform(x_train)

pipe = make_pipeline(column_trans, LogisticRegression(max_iter = 10000))
pipe.fit(x_train, y_train)

Wall time: 5.31 s


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'month', 'day_of_week',
                                                   'poutcome'])])),
                ('logisticregression', LogisticRegression(max_iter=10000))])

In [225]:
y_pred = pipe.predict(x_test)

In [226]:
confusion = confusion_matrix(y_test, y_pred)
confusion

array([[8832,  227],
       [ 723,  515]], dtype=int64)

In [227]:
print("recall - %.5f" %recall_score(y_test, y_pred))
print("precision - %.5f" %precision_score(y_test, y_pred))
print("f1 - %.5f" %f1_score(y_test, y_pred))
print("roc-auc score - %.5f" %roc_auc_score(y_test, y_pred))

recall - 0.41599
precision - 0.69407
f1 - 0.52020
roc-auc score - 0.69547


In [228]:
ohenc = []
ohenc.append(recall_score(y_test, y_pred))
ohenc.append(precision_score(y_test, y_pred))
ohenc.append(f1_score(y_test, y_pred))
ohenc.append(roc_auc_score(y_test, y_pred))
models_metrics['OneHotEncoder'] = ohenc

### MeanEncoder

One-hot-кодирование может сильно увеличивать количество признаков в датасете, что сказывается на памяти, особенно, если некоторый признак имеет большое количество значений. Эту проблему решает другой способ кодирования категориальных признаков — счётчики. Основная идея в том, что нам важны не сами категории, а значения целевой переменной, которые имеют объекты этой категории. Каждый категориальный признак мы заменим средним значением целевой переменной по всем объектам этой же категории:
$$
g_j(x, X) = \frac{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)][y_i = +1]}{\sum_{i=1}^{l} [f_j(x) = f_j(x_i)]}
$$

In [229]:
def MeanEncoder(data, categorical_feature, target_feature):
    df = data.copy()
    df[target_feature] = df[target_feature].map({'yes': 1, 'no': 0})
    for column_name in categorical_feature:
        mean_encoder = df.groupby([column_name])[target_feature].mean().to_dict()
        df[column_name] = df[column_name].map(mean_encoder)
    return df

In [230]:
%%time
data = MeanEncoder(df, categorical_features, 'y')

Wall time: 66 ms


In [231]:
X = data.drop(columns=['y'])
y = data['y']
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=777)

model = LogisticRegression(max_iter = 10000)
model.fit(x_train, y_train)

LogisticRegression(max_iter=10000)

In [232]:
y_pred = model.predict(x_test)

In [233]:
confusion = confusion_matrix(y_test, y_pred)
confusion

array([[8839,  220],
       [ 766,  472]], dtype=int64)

In [234]:
print("recall - %.5f" %recall_score(y_test, y_pred))
print("precision - %.5f" %precision_score(y_test, y_pred))
print("f1 - %.5f" %f1_score(y_test, y_pred))
print("roc-auc score - %.5f" %roc_auc_score(y_test, y_pred))

recall - 0.38126
precision - 0.68208
f1 - 0.48912
roc-auc score - 0.67849


In [235]:
menc = []
menc.append(recall_score(y_test, y_pred))
menc.append(precision_score(y_test, y_pred))
menc.append(f1_score(y_test, y_pred))
menc.append(roc_auc_score(y_test, y_pred))
models_metrics['MeanEncoder'] = menc

### Smooth Mean Encoder

Если переменная встечается всего несколько раз, тогда она будет слишком сильно коррелировать с целевой переменной,что впоследствии может оказать негативный результат на модель.В этом заключается главный минус MeanEncoder. По этой причине производится сглаживание счётчиков. Например, часто используются сглаживания средним по всей выборке:
$$
g_j(x, X) = \frac{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)][y_i = +1] + C \times global\_mean}{\sum_{i=1}^{\ell} [f_j(x) = f_j(x_i)] + C}
$$
где $global\_mean$ — доля объектов положительного класса в выборке, $C$ — параметр, определяющий степень сглаживания (например, можно использовать 10 или подобрать для каждого признака свой). 


In [236]:
def SmoothMeanEncoder(data, categorical_feature, target_feature, smooth):
    df = data.copy()
    df[target_feature] = df[target_feature].map({'yes': 1, 'no': 0})
    
    for column_name in categorical_feature:
        global_mean = df[target_feature].mean()
        variables_size = df.groupby([column_name]).size()
        variables_positive = df.groupby([column_name])[target_feature].sum()
        smooth_encoder = (variables_size * variables_positive + smooth * global_mean) / (variables_size + smooth)
        
        df[column_name] = df[column_name].map(smooth_encoder)
    return df

In [237]:
data = SmoothMeanEncoder(df,categorical_features, 'y', 0.5)

In [238]:
%%time
X = data.drop(columns=['y'])
y = data['y']
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=777)

model = LogisticRegression(max_iter = 10000)
model.fit(x_train, y_train)

Wall time: 3.34 s


LogisticRegression(max_iter=10000)

In [239]:
y_pred = model.predict(x_test)

In [240]:
confusion = confusion_matrix(y_test, y_pred)
confusion

array([[8824,  235],
       [ 758,  480]], dtype=int64)

In [241]:
print("recall - %.5f" %recall_score(y_test, y_pred))
print("precision - %.5f" %precision_score(y_test, y_pred))
print("f1 - %.5f" %f1_score(y_test, y_pred))
print("roc-auc score - %.5f" %roc_auc_score(y_test, y_pred))

recall - 0.38772
precision - 0.67133
f1 - 0.49155
roc-auc score - 0.68089


In [242]:
smenc = []
smenc.append(recall_score(y_test, y_pred))
smenc.append(precision_score(y_test, y_pred))
smenc.append(f1_score(y_test, y_pred))
smenc.append(roc_auc_score(y_test, y_pred))
models_metrics['SmoothMeanEncoder'] = smenc

### Results

In [248]:
for k,v in models_metrics.items():
    print(k)
    print(" recall - %.5f" %v[0])
    print(" precision - %.5f" %v[1])
    print(" f1 - %.5f" %v[2])
    print(" roc-auc score - %.5f" %v[3])
    

OrdinaryEncoder
 recall - 0.40630
 precision - 0.68810
 f1 - 0.51092
 roc-auc score - 0.69057
OneHotEncoder
 recall - 0.41599
 precision - 0.69407
 f1 - 0.52020
 roc-auc score - 0.69547
MeanEncoder
 recall - 0.38126
 precision - 0.68208
 f1 - 0.48912
 roc-auc score - 0.67849
SmoothMeanEncoder
 recall - 0.38772
 precision - 0.67133
 f1 - 0.49155
 roc-auc score - 0.68089


### RandomForest and OneHotEncoder

In [249]:
from sklearn.ensemble import RandomForestClassifier

In [252]:
y = df['y'].map({'yes': 1, 'no': 0})
X = df.drop(columns = 'y')

In [265]:
x_train, x_test, y_train, y_test = train_test_split(X,y, train_size = 0.75, random_state = 777)
transform = make_column_transformer((OneHotEncoder(), categorical_features), remainder = 'passthrough')
transform.fit_transform(x_train)

pipe = make_pipeline(transform, RandomForestClassifier(n_estimators = 5))
pipe.fit(x_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['job', 'marital',
                                                   'education', 'default',
                                                   'housing', 'loan', 'contact',
                                                   'month', 'day_of_week',
                                                   'poutcome'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(n_estimators=5))])

In [266]:
y_pred = pipe.predict(x_test)

In [267]:
print("recall - %.5f" %recall_score(y_test, y_pred))
print("precision - %.5f" %precision_score(y_test, y_pred))
print("f1 - %.5f" %f1_score(y_test, y_pred))
print("roc-auc score - %.5f" %roc_auc_score(y_test, y_pred))

recall - 0.44992
precision - 0.57840
f1 - 0.50613
roc-auc score - 0.70255
